In [14]:
import re
lines = []
Questions = []
Golds = []
QSplitted = []
GSplitted = []
col = []
Qid = []
lines = list(open("spider_multi_col_22000.txt", "r", encoding='utf-8').readlines())
# you may also want to remove whitespace characters like `\n` at the end of each line

Qids = list(open("spider_multi_col_dev.txt", "r", encoding='utf-8').readlines())

for i in range(0, len(Qids)):
    if i%4 == 1:
        Qid.append(Qids[i].strip('\n'))

for i in range(0, len(lines)):
    if i%3 == 0:
        Questions.append(lines[i])
        QSplitted.append(re.split(', and |, | and |\n',lines[i].strip("\n")))
    if i%3 == 1:
        Golds.append(lines[i])
        column = lines[i][7: lines[i].index("FROM")]
        col.append(re.split(' ,  | , ',column.strip(' '))) 

special = {"avg": "average ", "min": "minimum ", "max": "maximum ", 
         "sum": "sum of ", "count": "number of ", "Count": "number of ", "COUNT": "number of ", "count(DISTINCT": "number of different "}
        
for c in col:
    for i in range(0, len(c)):
        if "(" in c[i]:
            if 'count(DISTINCT' in c[i]:
                op = 'count(DISTINCT'
                att = c[i].strip('count(DISTINCT').strip(' ').strip(')')
            else:
                op = c[i][0:c[i].index('(')]
                att = c[i][c[i].index('(')+1:c[i].index(')')]
            if att == "*":
                c[i] = special[op.strip(' ')]
            else:
                c[i] = special[op.strip(' ')] + att

In [15]:

# Dynamic programming implementation of LCS problem 

# Returns length of LCS for X[0..m-1], Y[0..n-1] 
def lcs(X, Y, m, n): 
    L = [[0 for x in range(0, n+1)] for x in range(0, m+1)] 

    # Following steps build L[m+1][n+1] in bottom up fashion. Note 
    # that L[i][j] contains length of LCS of X[0..i-1] and Y[0..j-1] 
    for i in range(0, m+1): 
        for j in range(0, n+1): 
            if i == 0 or j == 0: 
                L[i][j] = 0
            elif X[i-1] == Y[j-1]: 
                L[i][j] = L[i-1][j-1] + 1
            else: 
                L[i][j] = max(L[i-1][j], L[i][j-1]) 
    
    # Following code is used to print LCS 
    index = L[m][n] 

    # Create a character array to store the lcs string 
    lcs = [""] * (index+1) 
    lcs[index] = "" 

    # Start from the right-most-bottom-most corner and 
    # one by one store characters in lcs[] 
    i = m 
    j = n 
    while i > 0 and j > 0: 

        # If current character in X[] and Y are same, then 
        # current character is part of LCS 
        if X[i-1] == Y[j-1]: 
            lcs[index-1] = X[i-1] 
            i-=1
            j-=1
            index-=1

        # If not same, then find the larger of two and 
        # go in the direction of larger value 
        elif L[i-1][j] > L[i][j-1]: 
            i-=1
        else: 
            j-=1
            
    return ''.join(lcs)
	#print ("LCS of " + X + " and " + Y + " is " + "".join(lcs) )

# Driver program 
X = "AGGTAB"
Y = "GXTXAYB"
m = len(X) 
n = len(Y) 
lcs(X, Y, m, n) 

# This code is contributed by BHAVYA JAIN 

'GTAB'

In [48]:
split = []
compare = []
count = 0
for i in range(0,len(col)):
    
    compare.append([len(QSplitted[i]), len(col[i])])
    #same.append(len(QSplitted[i]) == len(col[i]))
    if len(QSplitted[i]) == len(col[i]):
        tempfront = ''
        temprear = ''
        tempf = QSplitted[i][0].split(' ')
        tempr = QSplitted[i][len(col[i])-1].split(' ')
        #print(tempf)
        #print(tempr)
        
        ## Find the part of question in front of the first attribute
        frontScores = []
        frontCols = []
        for j in range(0, len(tempf)):
            #print(tempfront)
            tempfront = (tempf[len(tempf)-j-1] + ' ' + tempfront).strip(' ')
            #print('front = ' + tempfront)
            #print('col = ' + col[i][0])
            #print(lcs(tempfront, col[i][0], len(tempfront), len(col[i][0])))
            firstCol = col[i][0]
            frontScores.append(pow(len(lcs(tempfront.lower(), firstCol.lower(), len(tempfront), len(firstCol))), 2)/len(tempfront)/len(firstCol))
            frontCols.append(tempfront)
            #print(frontScores)
            #print(frontCols)
        frontIndex = frontScores.index(max(frontScores))
        frontCol = frontCols[frontIndex]
        #front = QSplitted[i][0].replace(frontCol, '', 1)
        front = ''.join(QSplitted[i][0].rsplit(frontCol, 1))
        ## Find the part of question after the last ribute
        rearScores = []
        rearCols = []
        for j in range(0, len(tempr)):
            #front = front + QSplitted[0]
            #print(temprear)
            temprear = (temprear + ' ' + tempr[j]).strip(' ')
            #print('rear = ' + temprear)
            #print('col = ' + col[i][len(col[i])-1])
            #print(lcs(temprear.lower(), col[i][len(col[i])-1], len(temprear), len(col[i][len(col[i])-1])))
            lastCol = col[i][len(col[i])-1]
            rearScores.append(pow(len(lcs(temprear.lower(), lastCol.lower(), len(temprear), len(lastCol))), 2)/len(temprear)/len(lastCol))
            rearCols.append(temprear)
            #print(rearScores)
            #print(rearCols)
        rearIndex = rearScores.index(max(rearScores))
        rearCol = rearCols[rearIndex]
        rear = QSplitted[i][len(QSplitted[i])-1].replace(rearCol, '', 1)        
        
        ## Split this question
        q = front + frontCol + rear
        g = 'SELECT ' + firstCol + Golds[i][Golds[i].index(' FROM'):]
        split.append([q, g.strip('\n'), Qid[i]])
        
        if len(QSplitted[i]) > 2:
            for h in range(1, len(QSplitted[i])-1):
                q = front + QSplitted[i][h] + rear
                g = 'SELECT ' + col[i][h] + Golds[i][Golds[i].index(' FROM'):]
                split.append([q, g.strip('\n'), Qid[i]])
    
        q = front + rearCol + rear
        g = 'SELECT ' + lastCol + Golds[i][Golds[i].index(' FROM'):]
        split.append([q, g.strip('\n'), Qid[i]])     
        ## count how many questions can be splitted using the algorithm
        count = count + 1
    else: split.append(col[i])
    

print(split)
print('Report:')
print('We split ' + str(count) + '/' + str(len(col)) + ' questions into a total of ' + str(len(split)) + ' questions. The percentage is about ' + str(round(100*(count)/(len(col)), 2)))
print('On average, the split questions have ' + str(round((len(split)-(len(col)-count))/count, 2)) + ' columns.')

[['Show name for all singers ordered by age from the oldest to the youngest.', 'SELECT name FROM singer ORDER BY age DESC', 'concert_singer'], ['Show country for all singers ordered by age from the oldest to the youngest.', 'SELECT country FROM singer ORDER BY age DESC', 'concert_singer'], ['Show age for all singers ordered by age from the oldest to the youngest.', 'SELECT age FROM singer ORDER BY age DESC', 'concert_singer'], ['What are the names for every singer in descending order of age?', 'SELECT name FROM singer ORDER BY age DESC', 'concert_singer'], ['What are the countries for every singer in descending order of age?', 'SELECT country FROM singer ORDER BY age DESC', 'concert_singer'], ['What are the ages for every singer in descending order of age?', 'SELECT age FROM singer ORDER BY age DESC', 'concert_singer'], ['What is the average of all singers from France?', "SELECT average age FROM singer WHERE country  =  'France'", 'concert_singer'], ['What is the minimum of all singers

In [201]:
notice = [0] * len(Golds)
for i in range(0, len(Golds)):
    gs = Golds[i].split(' ')
    if 'GROUP' in gs:
        gb = gs[gs.index('GROUP') + 2: gs.index('GROUP') + 3][0]
        #print(Golds[i])
        #print(gb)
        if gb in col[i] and len(col[i]) == 2 and 'number of ' in col[i]:
            print(Questions[i])
            print(Golds[i])
            print(col[i])
print(col)
print(QSplitted)
#same = []
        


Show all document ids and the number of paragraphs in each document. Order by document id.

SELECT document_id ,  count(*) FROM Paragraphs GROUP BY document_id ORDER BY document_id

['document_id', 'number of ']
Return the different document ids along with the number of paragraphs corresponding to each, ordered by id.

SELECT document_id ,  count(*) FROM Paragraphs GROUP BY document_id ORDER BY document_id

['document_id', 'number of ']
Which country has the most of TV Channels? List the country and number of TV Channels it has.

SELECT Country ,  count(*) FROM TV_Channel GROUP BY Country ORDER BY count(*) DESC LIMIT 1;

['Country', 'number of ']
What is the country with the most number of TV Channels and how many does it have?

SELECT Country ,  count(*) FROM TV_Channel GROUP BY Country ORDER BY count(*) DESC LIMIT 1;

['Country', 'number of ']
List the language used least number of TV Channel. List language and number of TV Channel.

SELECT LANGUAGE ,  count(*) FROM TV_Channel GROUP 

In [142]:
#print(same)
                
#testQ = []
#testG = []
#testQS = []
#testGS = []
#for i in range(0, 10):
    #print(i)
    #testQ.append(Questions[i])
    #testG.append(Golds[i])
    #testQS.append(QSplitted[i])
    #testGS.append(GSplitted[i])
    #print(Questions[i])
    #print(QSplitted[i])
    #print(Golds[i])
    #print(GSplitted[i])


0
